In [34]:
import requests
import zipfile
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from cycler import cycler

plt.rcParams['figure.dpi']     = 100
plt.rcParams['figure.figsize'] = 12,8
plt.rcParams['axes.prop_cycle'] = cycler('color', ['cadetblue','orangered','gold', 'navy'])

In [35]:
r = requests.get('https://arcgis.com/sharing/rest/content/items/a8c562ead9c54e13a135b02e0d875ffb/data')
zawartosc = r.content

with open('dane//dane.zip', 'wb') as plik:
    plik.write(zawartosc)


with zipfile.ZipFile('dane//dane.zip') as spaklowany_plik:
    spaklowany_plik.extractall(pathlib.Path('dane/pliki'))

lista_plikow = list(pathlib.Path.glob(pathlib.Path('dane/pliki'), '*.csv'))

kowidowe = []

for plik in lista_plikow:
    kowidowe.append(pd.read_csv(plik,encoding= 'ANSI', sep = ';'))   #windows-1250
    kowidowe[-1]['stan_rekordu_na'] = pd.to_datetime(str(plik)[11:19],format='%Y%m%d') - pd.Timedelta(1, unit='days')

wypisujemy nazwy kolumn w pliku z danymi archiwalnymi

In [65]:
arch_data0 = pd.read_csv(pathlib.Path('dane/mat_arch.csv'), sep = ';',decimal=' ', encoding= 'windows-1250')


ad = pd.DataFrame()
ad['wojewodztwo'] = ['Cały kraj'] * 266
ad['teryt'] = 't00'
ad['liczba_nowych_zakazen'] = arch_data0['Nowe przypadki'].astype('int32')
ad['liczba_ponownych_zakazen'] = 0
ad['liczba_wszystkich_zakazen'] = arch_data0['Nowe przypadki'].astype('int32')
ad['zgony'] = arch_data0['Zgony']
ad['zgony_w_wyniku_covid_bez_chorob_wspolistniejacych'] = np.NaN
ad['zgony_w_wyniku_covid_i_chorob_wspolistniejacych']   = np.NaN
ad['liczba_ozdrowiencow']  = arch_data0['Ozdrowieńcy (dzienna)']
ad['stan_rekordu_na']  = pd.to_datetime(arch_data0['Data'], format='%d.%m.%Y') - pd.Timedelta(days = 1)

ad

,wojewodztwo,teryt,liczba_nowych_zakazen,liczba_ponownych_zakazen,liczba_wszystkich_zakazen,zgony,zgony_w_wyniku_covid_bez_chorob_wspolistniejacych,zgony_w_wyniku_covid_i_chorob_wspolistniejacych,liczba_ozdrowiencow,stan_rekordu_na
0,Cały kraj,t00,0,0,0,0,NaN,NaN,0,2020-03-02
1,Cały kraj,t00,1,0,1,0,NaN,NaN,0,2020-03-03
2,Cały kraj,t00,0,0,0,0,NaN,NaN,0,2020-03-04
3,Cały kraj,t00,4,0,4,0,NaN,NaN,0,2020-03-05
4,Cały kraj,t00,1,0,1,0,NaN,NaN,0,2020-03-06
...,...,...,...,...,...,...,...,...,...,...
261,Cały kraj,t00,23975,0,23975,637,NaN,NaN,19914,2020-11-18
262,Cały kraj,t00,22464,0,22464,626,NaN,NaN,25275,2020-11-19
263,Cały kraj,t00,24213,0,24213,574,NaN,NaN,16896,2020-11-20
264,Cały kraj,t00,17856,0,17856,330,NaN,NaN,14897,2020-11-21


przejrzenie nazw kolumn w plikach w poszukiwaniu nie-duplikatów nazw kolumn

In [71]:
durnotka = []
for tab in kowidowe:
    if tab.shape[1] == 19:
        durnotka.append(pd.DataFrame(
            {
            'wojewodztwo': tab['wojewodztwo'],
            'teryt': tab['teryt'],
            'liczba_nowych_zakazen': tab['liczba_nowych_zakazen'],
            'liczba_ponownych_zakazen' : tab['liczba_ponownych_zakazen'],
            'liczba_wszystkich_zakazen': tab['liczba_wszystkich_zakazen'],
            'zgony' : tab['zgony'],
            'zgony_w_wyniku_covid_bez_chorob_wspolistniejacych' : tab['zgony_w_wyniku_covid_bez_chorob_wspolistniejacych'],
            'zgony_w_wyniku_covid_i_chorob_wspolistniejacych' : tab['zgony_w_wyniku_covid_i_chorob_wspolistniejacych'],
            'liczba_ozdrowiencow' : tab['liczba_ozdrowiencow'],
            'stan_rekordu_na' : tab['stan_rekordu_na']
            }))
    elif tab.shape[1] == 14 and 'liczba_ozdrowiencow' not in tab.columns:
         durnotka.append(pd.DataFrame(
            {
            'wojewodztwo': tab['wojewodztwo'],
            'teryt': tab['teryt'],
            'liczba_nowych_zakazen': tab['liczba_przypadkow'],
            'liczba_ponownych_zakazen' : 0,   ####################################################
            'liczba_wszystkich_zakazen': tab['liczba_przypadkow'],
            'zgony' : tab['zgony'],
            'zgony_w_wyniku_covid_bez_chorob_wspolistniejacych' : tab['zgony_w_wyniku_covid_bez_chorob_wspolistniejacych'],
            'zgony_w_wyniku_covid_i_chorob_wspolistniejacych' : tab['zgony_w_wyniku_covid_i_chorob_wspolistniejacych'],
            'liczba_ozdrowiencow' : 0,
            'stan_rekordu_na' : tab['stan_rekordu_na']
            }))

    elif tab.shape[1] == 15:
         durnotka.append(pd.DataFrame(
            {
            'wojewodztwo': tab['wojewodztwo'],
            'teryt': tab['teryt'],
            'liczba_nowych_zakazen': tab['liczba_przypadkow'],
            'liczba_ponownych_zakazen' : 0,   ####################################################
            'liczba_wszystkich_zakazen': tab['liczba_przypadkow'],
            'zgony' : tab['zgony'],
            'zgony_w_wyniku_covid_bez_chorob_wspolistniejacych' : tab['zgony_w_wyniku_covid_bez_chorob_wspolistniejacych'],
            'zgony_w_wyniku_covid_i_chorob_wspolistniejacych' : tab['zgony_w_wyniku_covid_i_chorob_wspolistniejacych'],
            'liczba_ozdrowiencow' : tab['liczba_ozdrowiencow'],
            'stan_rekordu_na' : tab['stan_rekordu_na']
            }))
durnotka.append(ad)
a = pd.concat(durnotka, ignore_index = True)
a



,wojewodztwo,teryt,liczba_nowych_zakazen,liczba_ponownych_zakazen,liczba_wszystkich_zakazen,zgony,zgony_w_wyniku_covid_bez_chorob_wspolistniejacych,zgony_w_wyniku_covid_i_chorob_wspolistniejacych,liczba_ozdrowiencow,stan_rekordu_na
0,CaĹ‚y kraj,t00,10141,0,10141,540.0,55.0,485.0,0,2020-11-23
1,dolnoĹ›lÄ…skie,t02,1033,0,1033,51.0,1.0,50.0,0,2020-11-23
2,kujawsko-pomorskie,t04,529,0,529,17.0,0.0,17.0,0,2020-11-23
3,lubelskie,t06,671,0,671,8.0,2.0,6.0,0,2020-11-23
4,lubuskie,t08,267,0,267,14.0,5.0,9.0,0,2020-11-23
...,...,...,...,...,...,...,...,...,...,...
12688,Cały kraj,t00,23975,0,23975,637.0,NaN,NaN,19914,2020-11-18
12689,Cały kraj,t00,22464,0,22464,626.0,NaN,NaN,25275,2020-11-19
12690,Cały kraj,t00,24213,0,24213,574.0,NaN,NaN,16896,2020-11-20
12691,Cały kraj,t00,17856,0,17856,330.0,NaN,NaN,14897,2020-11-21


<h4>Wyjściowa forma tabeli, to ta z plikow od 441 do końca, gdzie mamy 19 kolumn. <br>
będziemy starali sie zmienić oznaczenia kolumn z poprzednich plików na te 19ście </h4>


In [88]:
b = a.loc[a['teryt'] == 't00']
b.index = b['stan_rekordu_na'].drop(labels = ['wojewodztwo', 'teryt'], axis = 1)
sns.lineplot(data = b.rolling(7).mean(), y = 'zgony', label = '1')
sns.lineplot(data = b.rolling(7).mean(), x = 'stan_rekordu_na', y = 'zgony_w_wyniku_covid_bez_chorob_wspolistniejacych', label = '1')
sns.lineplot(data = b.rolling(7).mean(), x = 'stan_rekordu_na', y = 'zgony_w_wyniku_covid_i_chorob_wspolistniejacych', label = '1')
#sns.lineplot(data = a.loc[a['teryt'] == 't00'], x = 'stan_rekordu_na', y = 'liczba_ozdrowiencow', hue = 'teryt', label = '2')
plt.show()




ValueError: No axis named 1 for object type Series